# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
import pandas as pd

# Cargar el dataset
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

# 1. Crear un DataFrame que solo incluya clientes con un total_claim_amount mayor a $1,000 y respuesta "Yes"
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]

# Mostrar las primeras filas del DataFrame filtrado
print("Clientes con Total Claim Amount > $1,000 y respuesta 'Yes':")
print(filtered_df.head())

# 2. Analizar el promedio de total_claim_amount por tipo de póliza y género para clientes que respondieron "Yes"
# Filtrar el DataFrame para incluir solo clientes que respondieron "Yes"
yes_responders_df = df[df['Response'] == 'Yes']

# Crear una tabla pivote que muestra el promedio de total_claim_amount por Policy Type y Gender
pivot_avg_claim_by_policy_gender = pd.pivot_table(yes_responders_df, 
                                                  values='Total Claim Amount', 
                                                  index='Policy Type', 
                                                  columns='Gender', 
                                                  aggfunc='mean').round(2)

print("\nPromedio de Total Claim Amount por tipo de póliza y género:")
print(pivot_avg_claim_by_policy_gender)

# 3. Analizar el número total de clientes por estado y filtrar solo los estados con más de 500 clientes
# Contar el número de clientes por estado
state_counts = df['ST'].value_counts()

# Filtrar estados con más de 500 clientes
states_over_500 = state_counts[state_counts > 500]

print("\nNúmero total de clientes por estado (más de 500 clientes):")
print(states_over_500)

# 4. Encontrar el máximo, mínimo y mediano valor de Customer Lifetime Value por nivel educativo y género
# Agrupar por nivel educativo y género y calcular las estadísticas
clv_stats = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median'])

print("\nEstadísticas de Customer Lifetime Value por nivel educativo y género:")
print(clv_stats)

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
import pandas as pd

# Cargar el dataset
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

# Asegúrate de que la columna 'Date' esté en formato de fecha para extraer el mes
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Month'] = df['Date'].dt.to_period('M')

# 5. Número de pólizas vendidas por estado y mes
# Contar el número de pólizas vendidas por estado y mes
policy_count_by_state_month = df.groupby(['ST', 'Month']).size().unstack(fill_value=0)

print("Número de pólizas vendidas por estado y mes (meses como columnas y estados como filas):")
print(policy_count_by_state_month)

# 6. Número de pólizas vendidas por mes para los 3 estados con mayor número de pólizas vendidas
# Contar el número total de pólizas por estado
total_policies_by_state = df.groupby('ST').size()

# Obtener los 3 estados con más pólizas vendidas
top_states = total_policies_by_state.nlargest(3).index

# Filtrar el DataFrame para incluir solo los 3 estados principales
top_states_df = df[df['ST'].isin(top_states)]

# Contar el número de pólizas vendidas por estado y mes para los 3 estados principales
top_states_policy_count = top_states_df.groupby(['ST', 'Month']).size().unstack(fill_value=0)

print("\nNúmero de pólizas vendidas por mes para los 3 estados con más pólizas vendidas:")
print(top_states_policy_count)

# 7. Efecto de los diferentes canales de marketing en la tasa de respuesta del cliente
# Crear una tabla que muestra la tasa de respuesta por canal de marketing
response_rate_by_channel = df.groupby('Sales Channel')['Response'].apply(lambda x: (x == 'Yes').sum() / x.count()).reset_index()

# Renombrar columnas para mayor claridad
response_rate_by_channel.columns = ['Sales Channel', 'Response Rate']

print("\nTasa de respuesta por canal de marketing:")
print(response_rate_by_channel)